In [1]:
import pandas as pd
from os import listdir
from os.path import isfile, join

### Let's list all the files in the RawData folder

In [2]:
data_path = 'Datensatz/RawData'

file_names = [f for f in listdir(data_path) if isfile(join(data_path, f))]
sorted_file_names = sorted(file_names)

print(sorted_file_names)

['BSAD.csv', 'BSAK.csv', 'EKBE.csv', 'EKKO.csv', 'EKPO.csv', 'KNA1.csv', 'LFA1.csv', 'LFB1.csv', 'LFBK.csv', 'MAKT.csv', 'MARDH.csv', 'MBEWH.csv', 'RBKP.csv', 'REGUH.csv', 'REGUP.csv', 'RSEG.csv', 'T001.csv', 'T001K.csv', 'T001W.csv', 'USER_ADDR.csv', 'VBRK.csv', 'VBRP.csv']


### Function for Decistion Tree Evaluation

In [437]:
from sklearn.model_selection import KFold, train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import confusion_matrix, f1_score, precision_score 
import numpy as np



def evaluate_dt(X, y):
    
    TPR_all = []
    FPR_all = []
    FNR_all = []
    prec_all = []
    f1_all = []    

    dt = DecisionTreeClassifier(max_depth=2, random_state=11)
    kf = KFold(n_splits=10, random_state=12, shuffle=True)
    
    for train_index, test_index in kf.split(X):
        
        dt.fit(X.loc[train_index], y.loc[train_index])
        
        y_pred = dt.predict(X.loc[test_index])
        
        #scores
        cm = confusion_matrix(y.iloc[test_index], y_pred)
        
        TN = cm[0][0]
        FN = cm[1][0]
        TP = cm[1][1]
        FP = cm[0][1]
        
        
        TPR_all.append(float(TP)/(TP + FN))
        FPR_all.append(float(FP)/(FP + TN))
        FNR_all.append(float(FN)/(FN + TP))
        
        prec_all.append(precision_score(y.loc[test_index], y_pred))
        f1_all.append(f1_score(y.loc[test_index], y_pred))
        
    print('TPR: {}'.format(np.mean(TPR_all)))
    print('FPR: {}'.format(np.mean(FPR_all)))
    print('FNR: {}'.format(np.mean(FNR_all)))
    print('Prec: {}'.format(np.mean(prec_all)))
    print('F1: {}'.format(np.mean(f1_all)))
    
def evaluate_dt_split(X, y):
    
    TPR_all = []
    FPR_all = []
    FNR_all = []
    prec_all = []
    f1_all = []

    dt = DecisionTreeClassifier(max_depth=2, random_state=11)
    
        
    train_index, test_index = train_test_split(X.index.values.tolist(), random_state= 11)
        
    dt.fit(X.loc[train_index], y.loc[train_index])

    y_pred = dt.predict(X.loc[test_index])

    #scores
    cm = confusion_matrix(y.iloc[test_index], y_pred)

    TN = cm[0][0]
    FN = cm[1][0]
    TP = cm[1][1]
    FP = cm[0][1]

    
    TPR_all.append(float(TP)/(TP + FN))
    FPR_all.append(float(FP)/(FP + TN))
    FNR_all.append(float(FN)/(FN + TP))

    prec_all.append(precision_score(y.loc[test_index], y_pred, pos_label=1))
    f1_all.append(f1_score(y.loc[test_index], y_pred, pos_label=1))
        
        
    print('TN:{}'.format(TN))
    print('FN:{}'.format(FN))
    print('TP:{}'.format(TP))
    print('FP:{}'.format(FP))
    
    print('TPR: {}'.format(np.mean(TPR_all)))
    print('FPR: {}'.format(np.mean(FPR_all)))
    print('FNR: {}'.format(np.mean(FNR_all)))
    print('Prec: {}'.format(np.mean(prec_all)))
    print('F1: {}'.format(np.mean(f1_all)))  

### Case 1 and 2

In [440]:
lfa = pd.read_csv(join(data_path, 'LFA1.csv'), sep=';', encoding = 'ISO-8859-1')
lfa.shape

(10000, 13)

In [441]:
lfbk = pd.read_csv(join(data_path, 'LFBK.csv'), sep=';', encoding = 'ISO-8859-1', quoting=3)
lfbk.shape

(9997, 6)

In [442]:
m = pd.merge(lfa,lfbk, how='left', on=['LIFNR','MANDT'])
m.shape

(10000, 17)

In [443]:
lfb = pd.read_csv(join(data_path, 'LFB1.csv'), sep=';', encoding = 'ISO-8859-1', quoting=3)
lfb.shape

(12634, 5)

In [444]:
m2 = pd.merge(m,lfb[lfb['BUKRS']==1000], how='left', on=['LIFNR','MANDT'])
m2.shape

(10008, 20)

In [445]:
m2 = m2[pd.notnull(m2['CASE1']) & pd.notnull(m2['CASE2']) & pd.notnull(m2['ERDAT_y'])]
m2.shape

(9950, 20)

In [446]:
m2['NAME1'] = m2['NAME1'].astype(str)
m2['KOINH'] = m2['KOINH'].astype(str)

In [447]:
m2['FEAT'] = m2.apply(lambda x: x.KOINH[1:] in x.NAME1 , axis=1).astype(int)
m2['FEAT2'] = m2.apply(lambda x: x.ERDAT_x != x.ERDAT_y , axis=1).astype(int)
m2.shape

(9950, 22)

In [448]:
m2 = m2.reset_index(drop=True)
#m2 = m2.fillna(value=0)

In [449]:
X_cols = ['FEAT', 'FEAT2']
y_col1 = 'CASE1'
y_col2 = 'CASE2'

#### Prediction for Case 1

In [450]:
evaluate_dt(m2[X_cols], m2[y_col1])

TPR: 1.0
FPR: 0.0005072046451890455
FNR: 0.0
Prec: 0.9477272727272729
F1: 0.97227555053642


#### Prediction for Case 2

In [451]:
evaluate_dt(m2[X_cols], m2[y_col2])

TPR: 1.0
FPR: 0.00040578686284733004
FNR: 0.0
Prec: 0.9565934065934065
F1: 0.9769116809116809


### Case 4 (BMBTR != WRBTR)

In [452]:
regup = pd.read_csv(join(data_path, 'REGUP.csv'), sep=',', encoding = 'ISO-8859-1')
regup.shape

/gpfs/software/x86_64/anaconda/envs/anaconda431-py35/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (2,5) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


(101356, 16)

In [453]:
rseg = pd.read_csv(join(data_path, 'RSEG.csv'), sep=';', encoding = 'ISO-8859-1', quoting=3)
rseg.shape

(96012, 13)

In [454]:
#d  = pd.merge(regup, rseg, how='left', on=['MANDT','BELNR', 'BUKRS', 'BUZEI', 'GJAHR'])
d  = pd.merge(regup, rseg, how='left', on=['BELNR', 'GJAHR', 'BUKRS', 'MANDT'])

d.shape

(101356, 25)

In [455]:
#d['FEAT'] = d.apply(lambda x: float(x.DMBTR) == float(x.WRBTR), axis=1)

#d['FEAT2'] = d.apply(lambda x: float(x.DMBTR) - float(x.WRBTR), axis=1)

d['FEAT'] = d.apply(lambda x: np.isclose(x.DMBTR, x.WRBTR), axis=1)

d['FEAT2'] = d.apply(lambda x: np.isnan(x.WRBTR), axis=1)



In [456]:
X_cols = ['FEAT', 'FEAT2']
y_col = 'CASE4'

In [457]:
evaluate_dt(d[X_cols], d[y_col])

TPR: 1.0
FPR: 0.0
FNR: 0.0
Prec: 1.0
F1: 1.0


In [458]:
evaluate_dt_split(d[X_cols], d[y_col])

TN:24872
FN:0
TP:467
FP:0
TPR: 1.0
FPR: 0.0
FNR: 0.0
Prec: 1.0
F1: 1.0


### Case 5 (CPUTM Night time)

In [459]:
rbkp = pd.read_csv(join(data_path, 'RBKP.csv'), sep=';', encoding = 'ISO-8859-1')
rbkp.shape

(96012, 17)

In [487]:
import datetime 


def time_in_range(start,end, x):
    if start <= end:
        return start <= x <= end
    else:
        return start <= x or x <= end
    

frm = '%H:%M:%S'
start_time = datetime.time(18,0,0)
end_time = datetime.time(4,0,0)



In [478]:

rbkp['FEAT'] = rbkp.apply(lambda x: time_in_range(start_time, end_time, datetime.datetime.strptime(x.CPUTM, frm ).time()), axis=1)


In [485]:
X_cols = ['FEAT', 'GJAHR']
y_col = 'CASE5'

In [486]:
evaluate_dt(rbkp[X_cols], rbkp[y_col])

TPR: 1.0
FPR: 0.0
FNR: 0.0
Prec: 1.0
F1: 1.0


### Case 10 (USNAM)

In [ ]:
rbkp = pd.read_csv(join(data_path, 'RBKP.csv'), sep=';', encoding = 'ISO-8859-1')
rbkp.shape

In [489]:
rbkp[rbkp['CASE10']==0]['USNAM'].value_counts()

EWAGNER         6053
SFISCHER        6032
LWOLF           6017
AWEBER          6005
MBECKER         5999
HSCHNEIDER      5946
ESCHÄFER        5937
LKLEIN          5932
MMÜLLER         5930
ESCHMIDT        5917
LSCHULZ         5907
LHOFFMANN       5881
LBAUER          5859
LMEYER          5841
SRICHTER        5823
AKOCH           5817
LKLEIN            35
MMÜLLER            2
Name: USNAM, dtype: int64

In [493]:
rbkp[rbkp['CASE10']==1]['USNAM'].value_counts()

YWEINERT       121
BATCHUSER04    106
JSCHELLER      106
PRINTER         93
BUCHHALTUNG     73
BATCHUSER01     72
BATCHUSER02     72
LAGERUSER       72
BATCHUSER03     61
PRUEFER         56
AZUBI           55
DWEISE          54
FBODE           53
ASEIDL          46
KALTMANN        39
Name: USNAM, dtype: int64

In [495]:
err_names = rbkp[rbkp['CASE10']==1]['USNAM'].value_counts().index.values.tolist()
print(err_names)

['YWEINERT', 'BATCHUSER04', 'JSCHELLER', 'PRINTER', 'BUCHHALTUNG', 'BATCHUSER01', 'BATCHUSER02', 'LAGERUSER', 'BATCHUSER03', 'PRUEFER', 'AZUBI', 'DWEISE', 'FBODE', 'ASEIDL', 'KALTMANN']


In [498]:
rbkp['FEAT2'] = rbkp.apply(lambda x: x.USNAM in err_names, axis=1)


In [501]:
X_cols = ['FEAT2', 'GJAHR']
y_col = 'CASE10'

In [502]:
evaluate_dt(rbkp[X_cols], rbkp[y_col])

TPR: 1.0
FPR: 0.0
FNR: 0.0
Prec: 1.0
F1: 1.0


### Case 11  (LIFNR != SGTXT)

In [296]:
bsak = pd.read_csv(join(data_path, 'BSAK.csv'), sep=';', encoding = 'ISO-8859-1')
bsak.shape

/gpfs/software/x86_64/anaconda/envs/anaconda431-py35/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


(96062, 24)

#### Cast LIFNR column as string in order to create new column (True if SGTXT text contains LIFNR)

In [297]:
bsak['LIFNR'] = bsak['LIFNR'].astype(str)
bsak['FEAT'] = bsak.apply(lambda x: x.LIFNR  in x.SGTXT, axis=1)

In [298]:
num_cols = list(bsak.select_dtypes(include=['bool', 'int', 'float']).columns.values)

#### Select 3 predictor columns and CASE 11 as target variable

In [299]:
X_cols = ['FEAT', 'MONAT', 'BUZEI']
y_col = 'CASE11'

In [300]:
evaluate_dt(bsak[X_cols], bsak[y_col])

TPR: 1.0
FPR: 0.0
FNR: 0.0
Prec: 1.0
F1: 1.0


### Case 12 (NAME_TEXTC names)

In [254]:
addr = pd.read_csv(join(data_path, 'USER_ADDR.csv'), sep=';', encoding = 'ISO-8859-1')
addr.shape

(1011, 8)

In [255]:
wrong_names = ['Buchhaltung', 'Lageruser', 'Bachuser-01', 'Bachuser-02', 'Bachuser-03', 'Prüfer', 'Print User', 'Azubi']

In [256]:
addr['FEAT'] = addr.apply(lambda x: x.NAME_TEXTC  in wrong_names, axis=1)
addr = addr.reset_index(drop=True)


In [257]:
X_cols = ['FEAT', 'ID_USER']
y_col = 'CASE12'

In [282]:
evaluate_dt(addr[X_cols], addr[y_col])

TPR: 1.0
FPR: nan
FNR: 0.0
Prec: 0.2
F1: 0.2


/gpfs/software/x86_64/anaconda/envs/anaconda431-py35/lib/python3.5/site-packages/ipykernel/__main__.py:35: RuntimeWarning: invalid value encountered in true_divide


In [283]:
evaluate_dt_split(addr[X_cols], addr[y_col])

TN:252
FN:0
TP:1
FP:0
TPR: 1.0
FPR: 0.0
FNR: 0.0
Prec: 1.0
F1: 1.0
